In [5]:
import pandas as pd
import numpy as np
import os

os.getcwd()
os.chdir('C:/Users/Hao Liu/Documents/2_python')

def df_clean(data):
    newData = ''.join([i for i in data if not i.isdigit()])
    i = newData.find('(')
    if i>-1: newData = newData[:i]
    return newData.strip()

def answer_one():
    """import the first dataset"""
    df=pd.read_excel("Energy Indicators.xls",skiprows=17,skip_footer=38,na_values='...')
    energy=df.iloc[:,2:6]
    energy.columns=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy['Energy Supply']=energy['Energy Supply']*1000000.0

    energy['Country'] = energy['Country'].apply(df_clean)
    di = {"Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong"}
    energy.replace({"Country": di},inplace = True)


    """import the gdp dataset"""
    gdp=pd.read_csv("API_NY.GDP.MKTP.CD_DS2_en_csv_v2.csv",skiprows=3)
    di={"Korea, Rep.": "South Korea", 
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"}
    gdp.replace({"Country Name":di},inplace=True)
    gdp.rename(columns={'Country Name':'Country'},inplace=True)
    gdp10=gdp.loc[:,['Country','Country Code','Indicator Name','Indicator Code','2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

    """import the index dataset"""
    ScimEn=pd.read_excel("scimagojr.xlsx")
    ScimEn15=ScimEn[ScimEn["Rank"]<16]

    """merge"""
    df=pd.merge(pd.merge(energy,gdp10,on='Country'),ScimEn15,on='Country')
    df.set_index('Country',inplace=True)
    df=df.loc[:,['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]


    return df



In [10]:
df=answer_one()
df.shape


(15, 20)

In [ ]:
Top15 = answer_one()
cols=['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
test=Top15[cols].mean(axis=1).sort_values(ascending=False).rename('avgGDP')

In [ ]:
outer=pd.merge(pd.merge(energy,gdp,on='Country',how='outer'),ScimEn,on='Country',how='outer')

def answer_two():
    return len(outer)-len(df)


In [ ]:
def answer_three():
    Top15 = answer_one()
    cols=['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    test=Top15[cols].mean(axis=1).sort_values(ascending=False).rename('avgGDP')
    return test

In [ ]:
def answer_four():
    Top15 = answer_one()
    #find the No 6 index value, which is France
    No6=answer_three()
    No6name=No6[No6==No6[5]].index.values
    
    #slice the Top15 using the index value
    No6_df=Top15[Top15.index.values==No6name]
    
    return (No6_df['2015']-No6_df['2006'])

In [ ]:
def answer_five():
    Top15 = answer_one()
    return (Top15['Energy Supply per Capita'].mean())

In [ ]:
def answer_six():
    Top15 = answer_one()
    test=Top15.sort_values('% Renewable',ascending=False).iloc[0]
    return (test.name,test['% Renewable'])

In [ ]:
def answer_seven():
    Top15 = answer_one()
    Top15['ct-ratio']=Top15['Self-citations']/Top15['Citations'] 
    temp=Top15.sort_values(by='ct-ratio',ascending=False).iloc[0]
    return (temp.name,temp['ct-ratio'])

In [ ]:
def answer_eight():
    Top15 = answer_one()
    Top15['pop']=Top15['Energy Supply']/Top15['Energy Supply per Capita']
    temp=Top15.sort_values(by='pop',ascending=False).iloc[2]
    return (temp.name)

In [ ]:
def answer_nine():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15[['PopEst','Citable docs per Capita']].corr().iloc[0,1]
    return Top15[['PopEst','Citable docs per Capita']].corr().iloc[0,1]

In [ ]:
def answer_ten():
    Top15 = answer_one()
    median_renew=Top15['% Renewable'].median()  
    Top15['HighRenew']=np.where(Top15['% Renewable']>=median_renew,1,0)
    Top15.sort_values(by='Rank',ascending=True,inplace=True)
    return Top15['% Renewable']